In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

# Load Data

In [2]:
def load_scenario_data(h5_file_path, scenario_num):
    key = f"scenario_{scenario_num}/data"
    with pd.HDFStore(h5_file_path, mode='r') as store:
        if key not in store:
            return None  # Scenario not found
        df = store[key]
        metadata = store.get_storer(key).attrs.metadata
    return {"dataset": df, "metadata": metadata}

In [3]:
store_files = [
    "synthetic_data/RCT_0_5.h5",
    "synthetic_data/RCT_0_05.h5",
    "synthetic_data/e_X.h5",
    "synthetic_data/e_X_U.h5",
    "synthetic_data/e_X_info_censor.h5",
    "synthetic_data/e_X_U_info_censor.h5"
]

experiment_setups = {}

for path in store_files:
    base_name = os.path.splitext(os.path.basename(path))[0]  # e.g. RCT_0_5
    scenario_dict = {}
    for scenario in range(1, 11):
        try:
            result = load_scenario_data(path, scenario)
            if result is not None:
                scenario_dict[f"scenario_{scenario}"] = result
        except Exception as e:
            # Log or ignore as needed
            continue
    experiment_setups[base_name] = scenario_dict

In [4]:
experiment_setups['RCT_0_5']['scenario_1']['dataset'].head()

,id,observed_time,event,W,X1,X2,X3,X4,X5,U1,U2,T0,T1,T,C
0,0,0.054267,1,0,0.135488,0.887852,0.932606,0.445568,0.388236,0.151609,0.205535,0.054267,0.061394,0.054267,1.803019
1,1,0.732630,1,1,0.257596,0.657368,0.492617,0.964238,0.800984,0.597208,0.255785,0.228566,0.732630,0.732630,1.689546
2,2,0.162856,1,1,0.455205,0.801058,0.041718,0.769458,0.003171,0.370382,0.223214,0.176016,0.162856,0.162856,1.256329
3,3,0.050340,1,1,0.292809,0.610914,0.913027,0.300115,0.248599,0.038464,0.409829,0.381909,0.050340,0.050340,1.241777
4,4,0.524607,1,0,0.666392,0.987533,0.468270,0.123287,0.916031,0.342961,0.791330,0.524607,1.121968,0.524607,1.516613


In [5]:
experiment_repeat_setups = pd.read_csv("synthetic_data/idx_split.csv").set_index("idx")
experiment_repeat_setups

,random_idx0,random_idx1,random_idx2,random_idx3,random_idx4,random_idx5,random_idx6,random_idx7,random_idx8,random_idx9
idx,,,,,,,,,,
0,47390,5618,14210,46970,4203,16369,24535,45204,45725,45885
1,38566,46218,39045,7253,22759,34401,28889,38471,45822,37471
2,32814,20226,40012,4854,27351,39165,25359,14516,25717,29860
3,41393,39492,27153,19041,33009,19822,21243,41228,955,23901
4,12564,17823,48976,18458,22756,28169,45851,36620,29824,12711
...,...,...,...,...,...,...,...,...,...,...
49995,15948,39245,30779,48178,45056,4892,528,7486,31042,38267
49996,11102,29624,40779,3136,45904,41903,45682,36621,33204,38070
49997,16338,8986,19293,35651,10172,17947,38843,18310,2765,12581


# Run Experiments

In [30]:
from models_causal_survival.causal_survival_forest import CausalSurvivalForestGRF
from sklearn.metrics import mean_squared_error
import time
import pickle
import numpy as np
import pandas as pd

In [7]:
def prepare_data_split(dataset_df, experiment_repeat_setups, random_idx_col_list):
    split_results = {}

    for col in random_idx_col_list:
        random_idx = experiment_repeat_setups[col].values
        test_ids = random_idx[-5000:]
        train_ids = random_idx[:-5000]

        X_cols = [c for c in dataset_df.columns if c.startswith("X") and c[1:].isdigit()]
        
        train_df = dataset_df[dataset_df['id'].isin(train_ids)]
        test_df = dataset_df[dataset_df['id'].isin(test_ids)]

        X_train = train_df[X_cols].to_numpy()
        W_train = train_df["W"].to_numpy()
        Y_train = train_df[["observed_time", "event"]].to_numpy()

        X_test = test_df[X_cols].to_numpy()
        W_test = test_df["W"].to_numpy()
        Y_test = test_df[["observed_time", "event"]].to_numpy()

        CATE_test_true = (test_df["T1"] - test_df["T0"]).to_numpy()

        split_results[col] = (X_train, W_train, Y_train, X_test, W_test, Y_test, CATE_test_true)

    return split_results

In [ ]:
random_idx_col_list = ["random_idx0"]
failure_times_grid_size = 500
output_pickle_path = "causal_survival_forest_random_idx0_train_45000.pkl"

for setup_name, setup_dict in experiment_setups.items():
    for scenario_key in setup_dict:
        dataset_df = setup_dict[scenario_key]["dataset"]
        split_dict = prepare_data_split(dataset_df, experiment_repeat_setups, random_idx_col_list)

        if "result" not in experiment_setups[setup_name][scenario_key]:
            experiment_setups[setup_name][scenario_key]["result"] = {}

        start_time = time.time()

        for col in random_idx_col_list:
            X_train, W_train, Y_train, X_test, W_test, Y_test, CATE_test_true = split_dict[col]

            # Store placeholder for later population
            experiment_setups[setup_name][scenario_key]["result"][col] = {}

            # Train the model
            csf = CausalSurvivalForestGRF(failure_times_grid_size=failure_times_grid_size)
            csf.fit(X_train, W_train, Y_train)

            # Predict CATE
            CATE_test_pred = csf.predict_cate(X_test, W_test)

            # Save results
            experiment_setups[setup_name][scenario_key]["result"][col] = {
                "CATE_true": CATE_test_true,
                "CATE_Pred": CATE_test_pred,
                "CATE_MSE": mean_squared_error(CATE_test_true, CATE_test_pred)
            }

        end_time = time.time()

        experiment_setups[setup_name][scenario_key]["result"]["average"] = {
            "mean_CATE_MSE": np.mean([experiment_setups[setup_name][scenario_key]["result"][col]["CATE_MSE"]
                                      for col in random_idx_col_list]),
            "std_CATE_MSE": np.std([experiment_setups[setup_name][scenario_key]["result"][col]["CATE_MSE"]
                                     for col in random_idx_col_list]),
            "runtime": (end_time - start_time) / len(random_idx_col_list)
        }

        # Save progress to disk
        with open(output_pickle_path, "wb") as f:
            pickle.dump(experiment_setups, f)

    #     break
    # break
            


R[write to console]: Warning in (function (X, Y, W, D, W.hat = NULL, target = c("RMST", "survival.probability"),  :
R[write to console]: 
 
R[write to console]:  Estimated censoring probabilities go as low as: 0.0035 - an identifying assumption is that there exists a fixed positive constant M such that the probability of observing an event past the maximum follow-up time  is at least M (i.e. P(T > horizon | X) > M). This warning appears when M is less than 0.05, at which point causal survival forest can not be expected to deliver reliable estimates.



In [32]:
def summarize_experiment_results(experiment_setups):
    records = []

    for setup_name, setup_dict in experiment_setups.items():
        for scenario_key in setup_dict:
            avg_result = setup_dict[scenario_key].get("result", {}).get("average", {})
            mean_mse = avg_result.get("mean_CATE_MSE", np.nan)
            std_mse = avg_result.get("std_CATE_MSE", np.nan)
            runtime = avg_result.get("runtime", np.nan)

            records.append({
                "setup_name": setup_name,
                "scenario_key": scenario_key,
                "mean_CATE_mse": round(mean_mse, 4) if not pd.isna(mean_mse) else np.nan,
                "std_CATE_mse": round(std_mse, 4) if not pd.isna(std_mse) else np.nan,
                "runtime": round(runtime, 4) if not pd.isna(runtime) else np.nan
            })

    df = pd.DataFrame.from_records(records)
    return df

In [34]:
summary_df = summarize_experiment_results(experiment_setups)
summary_df

,setup_name,scenario_key,mean_CATE_mse,std_CATE_mse,runtime
0,RCT_0_5,scenario_1,0.7174,0.0,NaN
1,RCT_0_5,scenario_2,NaN,NaN,NaN
2,RCT_0_5,scenario_3,NaN,NaN,NaN
3,RCT_0_5,scenario_5,NaN,NaN,NaN
4,RCT_0_5,scenario_6,NaN,NaN,NaN
5,RCT_0_5,scenario_8,NaN,NaN,NaN
6,RCT_0_5,scenario_9,NaN,NaN,NaN
7,RCT_0_5,scenario_10,NaN,NaN,NaN
8,RCT_0_05,scenario_1,NaN,NaN,NaN
9,RCT_0_05,scenario_2,NaN,NaN,NaN
